In [7]:
import torch
import os
from pathlib import Path
import shutil

from modules.clearance import clearanceMLPModel

# 입력 파일 경로 (.ckpt 파일)
ckpt_path = "./checkpoint/mlp_all_120.ckpt"

# 출력 파일 경로 (.pt 파일) 
pt_path = "./checkpoint/mlp_all_120.pt"

print(f"입력 파일: {ckpt_path}")
print(f"출력 파일: {pt_path}")

# 입력 파일이 존재하는지 확인
if not os.path.exists(ckpt_path):
    print(f"오류: {ckpt_path} 파일을 찾을 수 없습니다.")
else:
    # 파일 크기 확인
    file_size = os.path.getsize(ckpt_path) / (1024 * 1024)  # MB 단위
    print(f"입력 파일 크기: {file_size:.2f} MB")

try:
    # .ckpt 파일 로드
    print(".ckpt 파일을 로드하는 중...")
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    
    print("체크포인트 키들:")
    for key in checkpoint.keys():
        print(f"  - {key}")
    
    # 모델 상태 딕셔너리 추출
    if 'state_dict' in checkpoint:
        model_state_dict = checkpoint['state_dict']
        print(f"\n모델 상태 딕셔너리 키 개수: {len(model_state_dict)}")
    else:
        print("\n'state_dict' 키를 찾을 수 없습니다. 전체 체크포인트를 저장합니다.")
        model_state_dict = checkpoint

    # 출력 디렉토리가 없으면 생성
    output_dir = os.path.dirname(pt_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"디렉토리 생성: {output_dir}")
    
    # .pt 파일로 저장
    print(".pt 파일로 저장하는 중...")
    torch.save(model_state_dict, pt_path)
    
    # 저장된 파일 크기 확인
    if os.path.exists(pt_path):
        pt_file_size = os.path.getsize(pt_path) / (1024 * 1024)  # MB 단위
        print(f"저장 완료!")
        print(f"출력 파일 크기: {pt_file_size:.2f} MB")
        
        # 크기 비교
        if os.path.exists(ckpt_path):
            ckpt_file_size = os.path.getsize(ckpt_path) / (1024 * 1024)
            compression_ratio = (1 - pt_file_size / ckpt_file_size) * 100
            print(f"압축률: {compression_ratio:.1f}%")

    # 변환된 .pt 파일이 제대로 로드되는지 확인
    print("\n변환된 .pt 파일 검증 중...")
    try:
        # 체크포인트에서 하이퍼파라미터 추출
        hparams = checkpoint['hyper_parameters']
        
        clearance_model = clearanceMLPModel(
            lr=hparams['lr'],
            dropout=hparams['dropout'], 
            model_name=hparams['model_name'],
            feature_length=hparams['feature_length'],
            act_func=hparams['act_func']
        )

        # .pt 파일에서 state dict 로드
        state_dict = torch.load(pt_path)

        #.pt파일 위치에 hparams.json 파일 생성
        import json
        with open(os.path.join(os.path.dirname(pt_path), 'hparams.json'), 'w') as f:
            json.dump(hparams, f)   
        
        # 모델에 state dict 적용
        clearance_model.load_state_dict(state_dict)
        print("모델 로드 성공!")
        
        # 모델을 평가 모드로 설정
        clearance_model.eval()
        print("모델이 정상적으로 동작합니다.")


        
    except Exception as e:
        print(f"모델 검증 중 오류 발생: {e}")


except Exception as e:
    print(f"오류 발생: {e}")
    raise



입력 파일: ./checkpoint/mlp_all_120.ckpt
출력 파일: ./checkpoint/mlp_all_120.pt
입력 파일 크기: 2205.49 MB
.ckpt 파일을 로드하는 중...
체크포인트 키들:
  - epoch
  - global_step
  - pytorch-lightning_version
  - state_dict
  - loops
  - callbacks
  - optimizer_states
  - lr_schedulers
  - hparams_name
  - hyper_parameters

모델 상태 딕셔너리 키 개수: 310
.pt 파일로 저장하는 중...
저장 완료!
출력 파일 크기: 735.16 MB
압축률: 66.7%

변환된 .pt 파일 검증 중...
모델 로드 성공!


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/root/miniconda3/envs/lightning/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/root/miniconda3/envs/lightning/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 